# Préparation données & librairies

In [1]:
import numpy as np
import pandas as pd

from data.DataManager import DataManager
from main.CrossRandomnessAnalysis import CrossRandomnessAnalysis
from data.DataCollectors import HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies, plot_3D
from utils.BlockConstructors import cross_overlapping_blocks, cross_non_overlapping_blocks 
from utils.MultiTester import CrossMultiTester


In [2]:
'''
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
'''

asset_pairs = ["BTCUSDT", "ETHUSDT"]

symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }

aggregation_levels = [1, 5, 10, 25, 50, 75, 100, 200, 500, 1000]
year, month = 2025, 2

In [4]:
historical_collector = HistoricalDataCollectorParquet(pairs=asset_pairs, year=year, month=month)
historical_collector.collect()

[SYSTEM] Processing BTCUSDT...
[SYSTEM] Parquet data already available for BTCUSDT → data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet → Skipping download.
[SYSTEM] Processing ETHUSDT...
[SYSTEM] Parquet data already available for ETHUSDT → data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet → Skipping download.


# Analyse des transactions

In [ ]:

tx_df = DataManager.summarize_transaction_characteristics(
    pairs=asset_pairs,
    symbols=symbols,
    year=year,
    month=month,
    aggregation_levels=aggregation_levels,
    max_interval_sec=1.0
)

tx_df

In [3]:
fig = DataManager.plot_transaction_counts_express(
    pairs=asset_pairs,
    symbols=symbols,
    year=year, month=month, day=None,
    aggregation_levels=aggregation_levels
)
fig.show()

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet


KeyError: "Column(s) ['log price'] do not exist"

#  Cross KL Divergence

In [3]:
alpha_eth = 5
# Least liquid pair is ETHUSDT
dm_eth = DataManager(['ETHUSDT'], symbols, year, month, None,
                     aggregation_level=alpha_eth)
n_blocks_eth = len(dm_eth.datasets['ETHUSDT']['symbol'])

# BTC is the most liquid pair, we use it to match the aggregation level of ETH to have the same number of blocks
alpha_btc = dm_eth.matching_aggregation_for("BTCUSDT", blocks=n_blocks_eth, year=year, month=month, day=None)

dm_btc = DataManager(['BTCUSDT'], symbols, year, month, None,
                      aggregation_level=alpha_btc)


[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet


In [4]:
blocks_eth = dm_eth.block_constructor(block_size=1, overlapping=False)['ETHUSDT']
blocks_btc = dm_btc.block_constructor(block_size=1, overlapping=False)['BTCUSDT']


We extract every aggregated symbol as its own row—no further grouping. This gives us the full, unbroken sequence of ordinal symbols at the chosen time‐tick granularity. With that sequence in hand, we can freely slide a window of size k over BTC and pair it with the very next ETH symbol (or construct disjoint windows) to perform our cross‐asset KL and entropy‐bias tests. By separating the notions of aggregation and windowing, we retain maximal flexibility and statistical validity.

## Test simple 

In [ ]:
k = 3   

symbols_btc = dm_btc.datasets['BTCUSDT']['symbol']
symbols_eth = dm_eth.datasets['ETHUSDT']['symbol']

cra = CrossRandomnessAnalysis(
    blocks_context=symbols_btc,
    blocks_target =symbols_eth,
    k=k,
    s=2,
    asset_context='BTCUSDT',
    alpha_context=alpha_btc,
    asset_target ='ETHUSDT',
    alpha_target =alpha_eth
)

freq_df    = cra.compute_cross_frequencies()
res_bias   = cra.entropy_bias_test()       # uses non-overlapping blocks 
res_kl     = cra.KL_divergence_test()      # uses overlapping blocks 


KeyError: 'symbol'

## Multi-test

In [ ]:
cmt = CrossMultiTester('BTCUSDT', 'ETHUSDT')

df_blocks = cmt.test_by_block_size(
    test='NP Statistic',
    max_block_size=10,
    alpha_context=5,
    year=year, month=month, day=None
)
display(df_blocks)

In [ ]:
df = cmt.test_by_aggregation_level(
    test='NP Statistic',
    aggregation_levels=[1,2,5,10,20,50],
    k=3,
    year=year, month=month, day=None
)

In [ ]:
cmt = CrossMultiTester('BTCUSDT', 'ETHUSDT')
result_3D = cmt.test_grid(
    test='NP Statistic',
    list_aggregations=[1,2,5,10,20,50],
    list_block_sizes=[1,2,3,4,5,6,7,8,9,10],
    year=year, month=month, day=None
)

plot_3D(result_3D, test='NP Statistic')

In [ ]:
df_grid = CrossMultiTester.to_dataframe_from_grid(
    result_3D,
    aggregation_levels=[1,2,5,10,20,50],
    block_sizes=[1,2,3,4,5,6,7,8,9,10],
    test_name='NP Statistic'
)
